In [2]:
import requests
import numpy as np
from scipy.stats import norm
import signal
from time import sleep


In [471]:
API_KEY = {'X-API-Key': 'besttrader'}
port = 9999
url = f'http://localhost:{port}/v1/' 
shutdown = False

In [481]:
class ApiException(Exception):
    pass

def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

#Rotman Functions
#Get current tick
def get_tick(session):
    resp = session.get(url+'case')
    if resp.status_code == 401:
        raise ApiException('API key incorrect')
    case = resp.json()
    return case['tick']

#Gets position and mid price of a security
def bid_ask(session):
    ticker = "ALGO"
    resp = session.get(url+f'securities?ticker={ticker}')
    sec = resp.json()
    sec = sec[0]
    #return [sec['position'],0.5*(sec['bid']+sec['ask'])]
    return (sec['bid'], sec['ask'])

def pm(session, bid_ask):
    ticker = "ALGO"
    params = {"ticker": "ALGO", "type":"MARKET", "quantity": 100, "price": bid_ask[0] + 0.01 , "action": "BUY" }
    resp = session.post('url'+f'orders?ticker={ticker}', params = params)
    sec = resp.json()
    #sec = sec[0]
    #return [sec['position'],0.5*(sec['bid']+sec['ask'])]
    #return sec
    print(sec)

def main():
    with requests.Session() as s:
        s.headers.update(API_KEY)
        lmt_sell_params = {'ticker': 'ALGO', 'type': 'LIMIT', 'quantity': 10,
        'price': 20.0, 'action': 'SELL'}
        resp = s.post('http://localhost:9999/v1/orders?ticker=ALGOtype=LIMIT?quantity=10?price=20.0?action=SELL')
        print(resp.json())

#strategy
#an auto-hedger


    
    
    
    
        
        
if __name__ == '__main__':
    signal.signal(signal.SIGINT, signal_handler)
    main()

{'code': 'BAD_REQUEST', 'message': 'Invalid ticker ALGOtype=LIMIT?quantity=10?price=20.0?action=SELL.'}


In [ ]:
#Setup
    #generate tickers
    option_tickers = ['RTM'+str(strike)+C_or_P for strike in range(45,55) for C_or_P in ('C','P')]
    underlying_ticker = 'RTM'
    #initial paramaters
    max_trades = 10 #per second
    delta_limit = 10000
    
    #begin per tick strategy
    
    tick = get_tick(session)
    
    while 1 < tick < 599:
        
        #reset trades
        
        num_trades = 0
        
        #Update time remaining
        
        tick = get_tick(session)
        
        T_t = (600-tick)/600/12
        
        #Update market info
        
        pos_mids = {option:pm(session,option) for option in option_tickers}
        pos_mids[underlying_ticker] = pm(session,underlying_ticker)
        underlying = pos_mids[underlying_ticker][1] #price of RTM
        
        #Update implied vols
        
        imp_vols = {option:imp_vol(underlying,strike(option),T_t,pos_mids[option][1],is_Call(option))
                    for option in option_tickers}
        
        #Update option deltas
        
        option_deltas = {option:delta(underlying, strike(option), T_t, imp_vols[option], is_Call(option))
                         for option in option_tickers}
        
        #determine portfolio delta
        
        port_delta = 100*sum([pos_mids[option][0]*option_deltas[option] for option in option_tickers])
        
        #now to hedge
        #desired position in RTM is -port_delta
        #I'm currently at pos_mids['RTM'][0] 
        #Need to buy or sell |pos_mids['RTM'][0] + port_delta| to reach 0 delta
        
        rtm_pos = pos_mids[underlying_ticker][0] 
        
        desired_pos = -port_delta
        
        if rtm_pos > desired_pos:
            direction = 'SELL'
        else:
            direction = 'BUY'
            
        if port_delta == 0:
            distance = abs(rtm_pos)
        else:
            distance = max(abs(rtm_pos-desired_pos)-delta_limit,0)
            
        if distance > 2*delta_limit:
            hedge = True
        elif tick%5 == 0:
            hedge = True
        else:
            hedge = False
            
        #place the orders
        while distance > 0 and hedge:
            
            if num_trades >= max_trades:
                break
            
            if distance > 10000:
                payload = {'ticker': underlying_ticker, 'type': 'MARKET', 'quantity': 10000, 'action': direction}
                session.post(url+'orders',params=payload)
                distance = distance - 10000
            else:
                payload = {'ticker': underlying_ticker, 'type': 'MARKET', 'quantity': distance, 'action': direction}
                session.post(url+'orders',params=payload)
                distance = 0
                
            num_trades = num_trades + 1
            
        sleep(1)